In [1]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [2]:
# Imports

from datasets import load_dataset

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DistilBertForSequenceClassification, DistilBertConfig

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [3]:
# load data
data = load_dataset("Vyshnev/phishing-data-classification")
data

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/697 [00:00<?, ?B/s]

(…)-00000-of-00001-e04e09853c3e1faa.parquet:   0%|          | 0.00/96.9k [00:00<?, ?B/s]

(…)-00000-of-00001-c921fd514ef8b30f.parquet:   0%|          | 0.00/21.1k [00:00<?, ?B/s]

(…)-00000-of-00001-5e065fd2492638da.parquet:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/450 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/450 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 2100
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 450
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 450
    })
})

In [4]:
# set up device
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [5]:
# load teacher model and tokenizer
model_path = "Vyshnev/bert-phishing-classifier_teacher"

tokenizer = AutoTokenizer.from_pretrained(model_path)
teacher_model = AutoModelForSequenceClassification.from_pretrained(model_path)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [6]:
# load student model
my_config = DistilBertConfig(n_heads=8, n_layers=4) #drop 4 heads per layer and 2 layers

student_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",
                                                                    config=my_config).to(device)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### tokenize text

In [7]:
#defince text preprocessing
def preprocess_function(example):
  return tokenizer(example['text'], padding='max_length', truncation=True)

#tokenize all dataset
tokenized_data = data.map(preprocess_function, batched=True)
tokenized_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/2100 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

### evaluation function

In [8]:
# Function to evaluate model performance
def evaluate_model(model, dataloader, device):
    model.eval()  # Set model to evaluation mode
    all_preds = []
    all_labels = []

    # Disable gradient calculations
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass to get logits
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Get predictions
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    # Calculate evaluation metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')

    return accuracy, precision, recall, f1

### train student model

In [9]:
# Function to compute distillation and hard-label loss
def distillation_loss(student_logits, teacher_logits, true_labels, temperature, alpha):
    # Compute soft targets from teacher logits
    soft_targets = nn.functional.softmax(teacher_logits / temperature, dim=1)
    student_soft = nn.functional.log_softmax(student_logits / temperature, dim=1)

    # KL Divergence loss for distillation
    distill_loss = nn.functional.kl_div(student_soft, soft_targets, reduction='batchmean') * (temperature ** 2)

    # Cross-entropy loss for hard labels
    hard_loss = nn.CrossEntropyLoss()(student_logits, true_labels)

    # Combine losses
    loss = alpha * distill_loss + (1.0 - alpha) * hard_loss

    return loss

In [10]:
# hyperparameters
batch_size = 32
lr = 1e-4
num_epochs = 5
temperature = 2.0
alpha = 0.5

# define optimizer
optimizer = optim.Adam(student_model.parameters(), lr=lr)

# create training data loader
dataloader = DataLoader(tokenized_data['train'], batch_size=batch_size)
# create testing data loader
test_dataloader = DataLoader(tokenized_data['test'], batch_size=batch_size)

In [12]:
# put student model in train mode
student_model.train()

# train model
for epoch in range(num_epochs):
    for batch in dataloader:
        # Prepare inputs
        teacher_model = teacher_model.to(device)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Disable gradient calculation for teacher model
        with torch.no_grad():
            teacher_outputs = teacher_model(input_ids, attention_mask=attention_mask)
            teacher_logits = teacher_outputs.logits

        # Forward pass through the student model
        student_outputs = student_model(input_ids, attention_mask=attention_mask)
        student_logits = student_outputs.logits

        # Compute the distillation loss
        loss = distillation_loss(student_logits, teacher_logits, labels, temperature, alpha)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} completed with loss: {loss.item()}")

    # Evaluate the teacher model
    teacher_accuracy, teacher_precision, teacher_recall, teacher_f1 = evaluate_model(teacher_model, test_dataloader, device)
    print(f"Teacher (test) - Accuracy: {teacher_accuracy:.4f}, Precision: {teacher_precision:.4f}, Recall: {teacher_recall:.4f}, F1 Score: {teacher_f1:.4f}")

    # Evaluate the student model
    student_accuracy, student_precision, student_recall, student_f1 = evaluate_model(student_model, test_dataloader, device)
    print(f"Student (test) - Accuracy: {student_accuracy:.4f}, Precision: {student_precision:.4f}, Recall: {student_recall:.4f}, F1 Score: {student_f1:.4f}")
    print("\n")

    # put student model back into train mode
    student_model.train()

Epoch 1 completed with loss: 0.13745981454849243
Teacher (test) - Accuracy: 0.8778, Precision: 0.8718, Recall: 0.8908, F1 Score: 0.8812
Student (test) - Accuracy: 0.8822, Precision: 0.8729, Recall: 0.8996, F1 Score: 0.8860


Epoch 2 completed with loss: 0.0758906826376915
Teacher (test) - Accuracy: 0.8778, Precision: 0.8718, Recall: 0.8908, F1 Score: 0.8812
Student (test) - Accuracy: 0.8911, Precision: 0.9412, Recall: 0.8384, F1 Score: 0.8868


Epoch 3 completed with loss: 0.1382150948047638
Teacher (test) - Accuracy: 0.8778, Precision: 0.8718, Recall: 0.8908, F1 Score: 0.8812
Student (test) - Accuracy: 0.8867, Precision: 0.8618, Recall: 0.9258, F1 Score: 0.8926


Epoch 4 completed with loss: 0.07069870829582214
Teacher (test) - Accuracy: 0.8778, Precision: 0.8718, Recall: 0.8908, F1 Score: 0.8812
Student (test) - Accuracy: 0.8978, Precision: 0.8797, Recall: 0.9258, F1 Score: 0.9021


Epoch 5 completed with loss: 0.09273621439933777
Teacher (test) - Accuracy: 0.8778, Precision: 0.8718,

In [13]:
# create testing data loader
validation_dataloader = DataLoader(tokenized_data['validation'], batch_size=8)

# Evaluate the teacher model
teacher_accuracy, teacher_precision, teacher_recall, teacher_f1 = evaluate_model(teacher_model, validation_dataloader, device)
print(f"Teacher (validation) - Accuracy: {teacher_accuracy:.4f}, Precision: {teacher_precision:.4f}, Recall: {teacher_recall:.4f}, F1 Score: {teacher_f1:.4f}")

# Evaluate the student model
student_accuracy, student_precision, student_recall, student_f1 = evaluate_model(student_model, validation_dataloader, device)
print(f"Student (validation) - Accuracy: {student_accuracy:.4f}, Precision: {student_precision:.4f}, Recall: {student_recall:.4f}, F1 Score: {student_f1:.4f}")

Teacher (validation) - Accuracy: 0.8844, Precision: 0.8811, Recall: 0.8889, F1 Score: 0.8850
Student (validation) - Accuracy: 0.9156, Precision: 0.9561, Recall: 0.8711, F1 Score: 0.9116


In [14]:
# Push to hub
from huggingface_hub import notebook_login

notebook_login()

In [15]:
student_model.push_to_hub("Vyshnev/bert-phishing-classifier_student")

model.safetensors:   0%|          | 0.00/211M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Vyshnev/bert-phishing-classifier_student/commit/4fe5d5e60758c8d1eabfbe25f0806a29dcd6488e', commit_message='Upload DistilBertForSequenceClassification', commit_description='', oid='4fe5d5e60758c8d1eabfbe25f0806a29dcd6488e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Vyshnev/bert-phishing-classifier_student', endpoint='https://huggingface.co', repo_type='model', repo_id='Vyshnev/bert-phishing-classifier_student'), pr_revision=None, pr_num=None)

### Infrencing

In [20]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_name = "Vyshnev/bert-phishing-classifier_student"  # or Vyshnev/bert-phishing-classifier_teacher
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Set up the device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/211M [00:00<?, ?B/s]

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-3): 4 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [23]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Define the model name and the correct tokenizer for it
student_model_name = "Vyshnev/bert-phishing-classifier_student"
tokenizer_name = "distilbert-base-uncased"

# Load the model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(student_model_name)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

# Manually define the label mapping
model.config.id2label = {0: "Safe", 1: "Not Safe"}
model.config.label2id = {"Safe": 0, "Not Safe": 1}

# Set up the device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Perform Inference
input_text = "google.com"
inputs = tokenizer(input_text, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

# Map prediction to label using the now-corrected config
predicted_label = model.config.id2label[predictions.item()]
print(f"The URL '{input_text}' is predicted as: {predicted_label}")

# --- Expected Output ---
# The URL 'google.com' is predicted as: Safe

The URL 'google.com' is predicted as: Safe


In [24]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the student model you want to fix
model_name = "Vyshnev/bert-phishing-classifier_student"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased") # Use the correct base tokenizer

# Add the correct label mappings to the model's configuration
model.config.id2label = {0: "Safe", 1: "Not Safe"}
model.config.label2id = {"Safe": 0, "Not Safe": 1}

# Log in to your Hugging Face account (if you haven't already)
# from huggingface_hub import notebook_login
# notebook_login()

# Push the updated configuration (and model) back to the Hub
model.push_to_hub(model_name, commit_message="Update config with id2label mapping")
tokenizer.push_to_hub(model_name, commit_message="Update config with id2label mapping") # Also push the tokenizer for consistency

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Vyshnev/bert-phishing-classifier_student/commit/f92872ea95cfe613587ed8de5c87c8ad61883f8d', commit_message='Update config with id2label mapping', commit_description='', oid='f92872ea95cfe613587ed8de5c87c8ad61883f8d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Vyshnev/bert-phishing-classifier_student', endpoint='https://huggingface.co', repo_type='model', repo_id='Vyshnev/bert-phishing-classifier_student'), pr_revision=None, pr_num=None)

In [25]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the student model from the Hub
student_model_name = "Vyshnev/bert-phishing-classifier_student"
model = AutoModelForSequenceClassification.from_pretrained(student_model_name)

# The student model uses the DistilBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Set up the device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-3): 4 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [26]:
input_text = "google.com"  # Replace with the URL you want to classify
inputs = tokenizer(input_text, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

# Map prediction to label using the model's configuration
predicted_label = model.config.id2label[predictions.item()]
print(f"The URL '{input_text}' is predicted as: {predicted_label}")

The URL 'google.com' is predicted as: Safe
